# Plotly
https://plot.ly/python/

Earlier I said that `seaborn` ***DID*** magic with `matplotlib`. In comparison, `plotly` ***IS*** magic.

Plotly is built on top of the Plotly JavaScript library (plotly.js). Plotly.py enables Python users to create beautiful interactive web-based visualizations that can be displayed in Jupyter notebooks, saved to standalone HTML files, or served as part of pure Python-built web applications using `Dash`.

## Using Plotly

Plotly has a very well designed API, so we're going to learn by doing (with a lot of `Shift+TAB`

In [2]:
import plotly.graph_objects as go # go

## Get the data ... again

In [71]:
import pandas as pd
temperatures = pd.read_csv("../data/global_temperatures/GlobalLandTemperaturesByCountry.csv", parse_dates=['dt'])
continents = pd.read_csv("../data/continents.csv")
countries = pd.read_csv("../data/countries.csv")
countries.drop(columns=['code'])
gdp = pd.read_csv("../data/2014_world_gdp_with_codes.csv")
temperatures = temperatures.merge(continents).merge(countries, left_on="Country", right_on="country").merge(gdp, left_on="Country", right_on="COUNTRY")

## Build Plotly Objects

Like in `matplotlib`, we use **figures** as the high-level object. Figures are populated with:

- **data**: comprised of instructions on what data is being plotted and how to arrange it, e.g. `data=[go.Bar(...)]`
- **layout**: styling and behavioural information for the figure (e.g. title, click events, etc.)
- Other elements are held in a convenient nested structure:
    - i.e. `go.Layout()` takes `annotations` as an argument, the object you pass in will be a list of `go.layout.Annotation()`

There's a graph object for everything, so how about a Bar graph? Let's dive right into this.

## EXERCISE

*Countries By Letter Of The Alphabet*

- Use the countries dataset to make a bar graph showing how many countries start with each letter of the alphabet
- The `.str` method of `pandas` will come in handy here
- Once you have something, click and drag, have fun with it. You've earned it!


- Bonus: can you add a **line** on top of the bars?

In [58]:
# Solution

# New column for first letter of country
countries['letter'] = countries.country.str[0]
# Count the countries per letter
countries_by_letter = countries.groupby('letter', as_index=False).country.count()

# Create the figure
fig = go.Figure(
    data=[
        go.Bar(
            x=countries_by_letter.letter,
            y=countries_by_letter.country,
            name="# of countries"
        ),
        go.Scatter(
            x=countries_by_letter.letter,
            y=countries_by_letter.country,
            name="a bad design choice",
        )
    ],
    layout=go.Layout(
        title="An Alphabet of Countries"
    )
)
fig

In [ ]:
# Your Solution

# Some data manipulation
# ...

# Build the figure
fig = go.Figure(
    data=[
        go.Bar(...)
    ]
)
fig

# It's like `matplotlib`, but better?

- We're still building figures from the bottom up, but the API is a bit nicer
- The big win is that we're getting modern interactive plots out of it
- Let's make some more

## EXERCISE

*Oh wait, that's not right!*

**Without** re-defining the objects:
- Edit the last figure to change the bar colour 
- Make the bar opacity 0.5
- If you have a line, change its style to "dash"

In [61]:
# Solution
fig.data[0].marker.color = "green"
fig.data[0].marker.opacity = 0.5
fig.data[1].line.dash = "dash"
fig

In [ ]:
# Your Solution

# change the fig object
fig # display the fig

# Maps

We could have done these with `matplotlib` and `seaborn`, but they are a lot more satisfying in `plotly`

## How it works:

- **data**: `go.Scattermapbox()`
    - lat, lon (to place the points)
    - mode (markers and/or text, likely don't want text if points are close together)
    - text (the actual text)
    - hoverinfo (probably 'text')

- **layout**: `mapbox=go.layout.Mapbox()`:
    - style: there are a number available for free, my favourites are:
        - stamen-watercolor (casual)
        - carto-positron (light theme)
        - carto-darkmatter (dark theme)

# EXERCISE

- Create a map of the world, with each country labelled

In [62]:
# Solution

lat = countries.lat
lon = countries.lon
locations_name = countries.country

data = [
    go.Scattermapbox(
        lat=lat,
        lon=lon,
        mode='markers',
        marker=dict(
            size=10,
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    )]

layout = go.Layout(
    title='Countries of the World',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=go.layout.Mapbox(
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='stamen-watercolor'
    ),
)

go.Figure(data=data, layout=layout)

In [ ]:
# Your Solution

# EXERCISE

*Now we're primed to take a shot at what we really came here for:*

- Calculate the change in average annual temperature over the last 50 years
- Create a Choropleth map showing this change for each country


- Note: Choropleth maps want a 3-letter country code


In [209]:
# Solution

# Calculate 50 year change
temperatures['year'] = temperatures.dt.dt.year
# Annual average
yearly_change = temperatures[(temperatures.year==1963) | (temperatures.year==2013)].groupby(["Country","year"], as_index=False).AverageTemperature.mean()
# Change
yearly_change['AverageTemperatureChange'] = yearly_change.groupby(["Country"], as_index=False).AverageTemperature.transform("diff")
# Drop the NaNs we just created
yearly_change.dropna(inplace=True)
# Join Country Code
temperature_slice=yearly_change.merge(temperatures[["Country","Code"]].drop_duplicates())


fig = go.Figure(
    data=[go.Choropleth(
        z=temperature_slice.AverageTemperatureChange,
        locations=temperature_slice.Code,
        text=temperature_slice.Country,
        colorscale="RdBu",
        reversescale=True
)])
fig

In [198]:
# Your Solution

# Calculate 50 year change

# Plot map
fig = go.Figure(
    data=[go.Choropleth()]
)
fig